# Title

Importing relevant libraries

In [2]:
import numpy as num
import scipy as sci
import pandas as pd
import matplotlib
import nltk
import sklearn as skl
import xgboost
import gensim
import json
import re
import string

Taking a fraction of the entire dataset

In [3]:
trainingAmount = 400000
testAmount = 100000

with open('review.json') as raw:
    with open('training.json', 'w') as trainingSet: 
        for x in range(trainingAmount):
            line = raw.readline()
            trainingSet.write(line)
    with open('test.json', 'w') as testSet: 
        for x in range(trainingAmount, trainingAmount+testAmount):
            line = raw.readline()
            testSet.write(line)

In [4]:
df_train = pd.read_json('training.json', lines=True)
df_test = pd.read_json('test.json', lines=True)

In [5]:
df_test = df_test.drop("review_id", axis=1).drop("business_id", axis=1).drop("user_id", axis=1).drop("date", axis=1)
df_test = df_test.reindex(['text','stars','useful','funny','cool'], axis=1)

df_train = df_train.drop("review_id", axis=1).drop("business_id", axis=1).drop("user_id", axis=1).drop("date", axis=1)
df_train = df_train.reindex(['text','stars','useful','funny','cool'], axis=1)

df_train['length'] = df_train['text'].apply(len)
df_test['length'] = df_test['text'].apply(len)

In [6]:
def clean(text):
     # Empty question
    if type(text) != str:
        return ''
    
#     nltk.corpus.stopwords.words('english')
    
    
    text = re.sub("I\'ll", "I will", text)
    text = re.sub("I\'m", "I am", text)

    text = text.lower()
    
    text = re.sub(" what\'s ", " what is ", text)
    text = re.sub(" that\'s ", " that is ", text)
    
    text = re.sub("\'s", "", text)
    
    
    text = re.sub(" whats ", " what is ", text)
    text = re.sub(" thats ", " that is ", text)
    text = re.sub("\'ve", " have ", text)
    text = re.sub("n\'t", ' not', text)

    text = re.sub('[' + string.punctuation + ']', '', text)
    
    # remove comma between numbers, i.e. 15,000 -> 15000
    text = re.sub('(?<=[0-9])\,(?=[0-9])', "", text)
    
    # Return a list of words
    return text

In [12]:
df_train.describe()

,stars,useful,funny,cool
count,400000.000000,400000.000000,400000.000000,400000.000000
mean,3.730208,1.300900,0.451282,0.549493
std,1.455814,2.956285,1.676008,2.029620
min,1.000000,0.000000,0.000000,0.000000
25%,3.000000,0.000000,0.000000,0.000000
50%,4.000000,0.000000,0.000000,0.000000
75%,5.000000,1.000000,0.000000,0.000000
max,5.000000,135.000000,85.000000,108.000000


Cleaning the data

In [7]:
df_train['text'] = df_train['text'].apply(clean)
df_test['text'] = df_test['text'].apply(clean)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

train = df_train['text'].values
test = df_test['text'].values

vectorizer = CountVectorizer()
vectorizer.fit_transform(train)

trainQvec = scipy.sparse.hstack([trainQ1vec, trainQ2vec])
trainLabels = df_train['is_duplicate'].values

In [17]:
df_test

,text,stars,useful,funny,cool,length
0,very good sushi extremely nice staff and timel...,5,0,0,0,93
1,not sure if this should be listed seperately b...,4,0,0,0,432
2,so there are 2 hard rock cafes in las vegas on...,3,0,0,0,1631
3,good restaurant but they always always over co...,3,0,0,0,179
4,overall very average compared to the hype not ...,3,1,0,0,398
5,this is such a great spot in the south hills s...,5,0,0,0,149
6,i have only ever come here for lunch which ru...,5,0,0,0,219
7,solid three stars the pizza slices suckare dry...,3,0,0,2,289
8,we ordered delivery here prompt and quick serv...,5,2,0,1,923
9,i do not like to leave low reviews but this wa...,1,1,0,0,1284
